In [4]:
%pip install pyarrow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 24.3 MB/s  0:00:01 eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Run first in every notebook
import os, sys, pathlib
if pathlib.Path().resolve().name == "notebooks":
    os.chdir("..")                         # move CWD to project root
sys.path.insert(0, os.path.abspath("src")) # make `src/` importable
print("CWD:", os.getcwd())

CWD: /Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis


In [8]:
import sys, os
print(sys.executable)
print(sys.path[:3])


/Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis/.venv/bin/python
['/Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis/src', '/Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis/src', '/opt/homebrew/Cellar/python@3.11/3.11.13/Frameworks/Python.framework/Versions/3.11/lib/python311.zip']


In [15]:
import importlib, inspect, src.fetch_data
import shutil, pathlib

# clear any stale bytecode
shutil.rmtree(pathlib.Path("src/__pycache__"), ignore_errors=True)

# reload the module
importlib.reload(src.fetch_data)

# sanity check: show the new function source
print(src.fetch_data.__file__)
print(inspect.getsource(src.fetch_data.save_artifacts))

/Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis/src/fetch_data.py
def save_artifacts(df: pd.DataFrame, out_dir: str | Path = "data/derived"):
    """
    CSV-only saver to avoid pyarrow/Parquet issues.
    """
    ensure_dir(out_dir)
    Path(out_dir, "trials_clean.csv").write_text(df.to_csv(index=False))
    by_pid = df.groupby("participantId").agg(
        n=("correct_int", "size"),
        acc=("correct_int", "mean"),
        rt_med=("rt", "median"),
    ).reset_index()
    by_pid.to_csv(Path(out_dir, "participant_summary.csv"), index=False)
    return {"parquet_written": False}



In [16]:
from src.fetch_data import load_cleaned_trials, summarize, save_artifacts

CLEANED = "data/exports/trials_20250811T043446Z_cleaned.csv"
df = load_cleaned_trials(CLEANED)
summary = summarize(df); summary
save_artifacts(df, out_dir="data/derived")


/Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis/src/fetch_data.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  out["acc_by_p_bin"] = df.groupby(pbin)["correct_int"].mean().to_dict()
/Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis/src/fetch_data.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  out["rt_median_by_abs_delta"] = df.groupby(dbin)["rt"].median().to_dict()
/Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis/src/fetch_data.py:24: FutureWarning: The default of observed=False is deprecated and will be changed to Tru

{'parquet_written': False}

In [10]:
from src.plots import psychometric, accuracy_vs_absdelta, rt_vs_absdelta
fig1 = psychometric(df)
fig2 = accuracy_vs_absdelta(df)
fig3 = rt_vs_absdelta(df)
fig1, fig2, fig3

Matplotlib is building the font cache; this may take a moment.
/Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis/src/plots.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("delta_bin")["is_higher"].mean().reset_index()
/Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis/src/plots.py:42: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby("abs_delta_bin")["correct_int"].agg(["mean","size"]).reset_index()
/Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis/src/plots.py:64: FutureWarning: The default of observed=F

('data/figures/psychometric_delta.png',
 'data/figures/accuracy_vs_absdelta.png',
 'data/figures/rt_vs_absdelta.png')

In [11]:
import pandas as pd
pd.read_csv("data/derived/participant_summary.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/derived/participant_summary.csv'